# Grabing data from LastFM API

In [1]:
import lastfm

lastfm.import_historic_data(file_path='lastfm_data', initial_year=2017,end_year=2021)

saved lastfm_data2016.csv


'files saved correctly at lastfm_data'

In [2]:
import pandas
lasffm_files = 'lastfm_data*.csv'
lastfm_df = pandas.read_csv(lasffm_files)
lastfm_df.head(10)

,artist_name,track_name,loved_track,date_uts,date
0,George Harrison,Wah-Wah (2020 Mix),0,NaN,NaT
1,Queensrÿche,Live Evolution - Hitnf/Q2K Suite: Sacred Ground,0,1233353637,2009-01-30 20:13:57
2,Queensrÿche,Live Evolution - Hitnf/Q2K Suite: Liquid Sky,0,1233353337,2009-01-30 20:08:57
3,Queensrÿche,Live Evolution - Empire/Promised Land Suite: E...,0,1233351743,2009-01-30 19:42:23
4,Queensrÿche,Live Evolution - Empire/Promised Land Suite: D...,0,1233351492,2009-01-30 19:38:12
5,Queensrÿche,Live Evolution - Empire/Promised Land Suite: I...,0,1233351248,2009-01-30 19:34:08
6,Queensrÿche,Live Evolution - Rage/Mindcrime Suite: Eyes of...,0,1233350871,2009-01-30 19:27:51
7,Queensrÿche,Live Evolution - Rage/Mindcrime Suite: My Empt...,0,1233350790,2009-01-30 19:26:30
8,Queensrÿche,Live Evolution - Rage/Mindcrime Suite: I Don't...,0,1233350527,2009-01-30 19:22:07
9,Queensrÿche,Live Evolution - EP/Warning Suite: London,0,1233350281,2009-01-30 19:18:01


In [1]:
import pandas
lasffm_files = 'lastfm_data2015.csv'
lastfm_df = pandas.read_csv(lasffm_files)
lastfm_df.head(10)

,Unnamed: 0,artist_name,track_name,loved_track,date_uts,date
0,0,George Harrison,Ballad of Sir Frankie Crisp (Let It Roll) [202...,0,NaN,NaN
1,1,Buffalo Springfield,Nowadays Clancy Can't Even Sing - Remastered,0,1.450313e+09,2015-12-16 22:45:40
2,2,Buffalo Springfield,Can't Keep Me Down - Originally Unreleased Demo,0,1.450313e+09,2015-12-16 22:43:30
3,3,Buffalo Springfield,Sad Memory - Originally Unreleased Demo,0,1.450313e+09,2015-12-16 22:40:36
4,4,Buffalo Springfield,We'll See - Originally Unreleased Demo,0,1.450313e+09,2015-12-16 22:36:25
5,5,Buffalo Springfield,Neighbor Don't You Worry - Originally Unreleas...,0,1.450312e+09,2015-12-16 22:33:54
6,6,Buffalo Springfield,Baby Don't Scold Me - Originally Unreleased Demo,0,1.450312e+09,2015-12-16 22:31:46
7,7,Buffalo Springfield,I'm Your Kind Of Guy - Originally Unreleased Demo,0,1.450312e+09,2015-12-16 22:30:38
8,8,Buffalo Springfield,Flying On The Ground Is Wrong - Originally Unr...,0,1.450312e+09,2015-12-16 22:27:27
9,9,Buffalo Springfield,Out Of My Mind - Originally Unreleased Demo Box,0,1.450312e+09,2015-12-16 22:24:42
